<a href="https://colab.research.google.com/github/ismachy/Introduction-to-ML/blob/main/HW_5_Q_02_a_b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [2]:
housing = pd.read_csv('Housing.csv')
housing

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,yes,no,yes,no,no,2,no,unfurnished
541,1767150,2400,3,1,1,no,no,no,no,no,0,no,semi-furnished
542,1750000,3620,2,1,1,yes,no,no,no,no,0,no,unfurnished
543,1750000,2910,3,1,1,no,no,no,no,no,0,no,furnished


In [3]:
num_vars = ['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'parking']
df_new = housing[num_vars]

In [4]:
scaler = MinMaxScaler()
df_new[num_vars] = scaler.fit_transform(df_new[num_vars])
df_new.head()

<ipython-input-4-e90be4113ded>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new[num_vars] = scaler.fit_transform(df_new[num_vars])


,price,area,bedrooms,bathrooms,stories,parking
0,1.000000,0.396564,0.6,0.333333,0.666667,0.666667
1,0.909091,0.502405,0.6,1.000000,1.000000,1.000000
2,0.909091,0.571134,0.4,0.333333,0.333333,0.666667
3,0.906061,0.402062,0.6,0.333333,0.333333,1.000000
4,0.836364,0.396564,0.6,0.000000,0.333333,0.666667


In [5]:
X = df_new.iloc[:, 1:6].values
Y = df_new.iloc[:, 0].values

In [6]:
X = torch.tensor(X)
Y = torch.tensor(Y)

In [7]:
def model(X, W1, W2, W3, W4, W5, B):
    return W5*X[:,4] + W4*X[:,3] + W3*X[:,2] + W2*X[:,1] + W1*X[:,0] + B

In [8]:
def loss_fn(Y_p, Y):
    squared_diffs = (Y_p - Y)**2
    return squared_diffs.mean()

In [9]:
W1 = torch.ones(())
W2 = torch.ones(())
W3 = torch.ones(())
W4 = torch.ones(())
W5 = torch.ones(())
B = torch.zeros(())

In [10]:
n_samples = X.shape[0]
n_val = int(0.2 * n_samples)
shuffled_indices = torch.randperm(n_samples)
train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

In [11]:
train_X = X[train_indices]
train_Y = Y[train_indices]
val_X = X[val_indices]
val_Y = Y[val_indices]

In [12]:
def training_loop(n_epochs, optimizer, params, train_X, val_X, train_Y, val_Y):

    for epoch in range(1, n_epochs + 1):
        train_Y_p = model(train_X, *params)
        train_loss = loss_fn(train_Y_p, train_Y)

        val_Y_p = model(val_X, *params)
        val_loss = loss_fn(val_Y_p, val_Y)

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        if epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
                  f" Validation loss {val_loss.item():.4f}")

    return params

In [13]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-1
optimizer = optim.Adam([params], lr=learning_rate)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    params = params,
    train_X = train_X,
    val_X = val_X,
    train_Y = train_Y,
    val_Y = val_Y)

Epoch 500, Training loss 0.0106, Validation loss 0.0152
Epoch 1000, Training loss 0.0106, Validation loss 0.0152
Epoch 1500, Training loss 0.0106, Validation loss 0.0152
Epoch 2000, Training loss 0.0106, Validation loss 0.0152
Epoch 2500, Training loss 0.0106, Validation loss 0.0152
Epoch 3000, Training loss 0.0106, Validation loss 0.0152
Epoch 3500, Training loss 0.0106, Validation loss 0.0152
Epoch 4000, Training loss 0.0106, Validation loss 0.0152
Epoch 4500, Training loss 0.0106, Validation loss 0.0152
Epoch 5000, Training loss 0.0106, Validation loss 0.0152


tensor([0.4028, 0.0860, 0.3171, 0.1131, 0.0988, 0.0446], requires_grad=True)

In [14]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.Adam([params], lr=learning_rate)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    params = params,
    train_X = train_X,
    val_X = val_X,
    train_Y = train_Y,
    val_Y = val_Y)

Epoch 500, Training loss 0.0125, Validation loss 0.0174
Epoch 1000, Training loss 0.0108, Validation loss 0.0155
Epoch 1500, Training loss 0.0106, Validation loss 0.0152
Epoch 2000, Training loss 0.0106, Validation loss 0.0152
Epoch 2500, Training loss 0.0106, Validation loss 0.0152
Epoch 3000, Training loss 0.0106, Validation loss 0.0152
Epoch 3500, Training loss 0.0106, Validation loss 0.0152
Epoch 4000, Training loss 0.0106, Validation loss 0.0152
Epoch 4500, Training loss 0.0106, Validation loss 0.0152
Epoch 5000, Training loss 0.0106, Validation loss 0.0152


tensor([0.4028, 0.0860, 0.3171, 0.1131, 0.0988, 0.0446], requires_grad=True)

In [15]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-3
optimizer = optim.Adam([params], lr=learning_rate)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    params = params,
    train_X = train_X,
    val_X = val_X,
    train_Y = train_Y,
    val_Y = val_Y)

Epoch 500, Training loss 0.1228, Validation loss 0.1452
Epoch 1000, Training loss 0.0501, Validation loss 0.0562
Epoch 1500, Training loss 0.0333, Validation loss 0.0372
Epoch 2000, Training loss 0.0223, Validation loss 0.0257
Epoch 2500, Training loss 0.0166, Validation loss 0.0203
Epoch 3000, Training loss 0.0139, Validation loss 0.0182
Epoch 3500, Training loss 0.0127, Validation loss 0.0174
Epoch 4000, Training loss 0.0119, Validation loss 0.0168
Epoch 4500, Training loss 0.0114, Validation loss 0.0163
Epoch 5000, Training loss 0.0110, Validation loss 0.0158


tensor([ 0.4293,  0.2354,  0.2750,  0.0915,  0.0918, -0.0108],
       requires_grad=True)

In [16]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-4
optimizer = optim.Adam([params], lr=learning_rate)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    params = params,
    train_X = train_X,
    val_X = val_X,
    train_Y = train_Y,
    val_Y = val_Y)

Epoch 500, Training loss 0.9726, Validation loss 1.1010
Epoch 1000, Training loss 0.7813, Validation loss 0.8885
Epoch 1500, Training loss 0.6203, Validation loss 0.7090
Epoch 2000, Training loss 0.4860, Validation loss 0.5588
Epoch 2500, Training loss 0.3756, Validation loss 0.4347
Epoch 3000, Training loss 0.2864, Validation loss 0.3338
Epoch 3500, Training loss 0.2161, Validation loss 0.2536
Epoch 4000, Training loss 0.1626, Validation loss 0.1919
Epoch 4500, Training loss 0.1234, Validation loss 0.1461
Epoch 5000, Training loss 0.0963, Validation loss 0.1137


tensor([ 0.6163,  0.6156,  0.5926,  0.5957,  0.5944, -0.3696],
       requires_grad=True)